In [7]:
import pandas as pd
from langdetect import detect
from SPARQLWrapper import SPARQLWrapper, JSON
import re
import numpy as np
import sys
import os
import time
from nltk.tag import pos_tag
import urllib.parse

In [ ]:
#             '{?resource rdfs:label ?name1 FILTER regex(?name1,'  + '"' + str(cell_content) + '", "i") . ' + \

cell_content = 'Down to Earth'
film_year = 2001
start = time.time()
query = 'SELECT ?resource WHERE {' + \
            '{?resource rdfs:label ?name1 FILTER regex(?name1,'  + '"^' + str(cell_content) + '$", "i") . ' + \
            '?resource rdf:type dbo:Film . }' + \
        'UNION ' + \
            '{?resource rdfs:label ?name1 FILTER regex(?name1,'  + '"^The ' + str(cell_content) + '$", "i") . ' + \
            '?resource rdf:type dbo:Film . }' + \
        'UNION ' + \
            '{?resource rdfs:label '  + '"' + str(cell_content) + '" . ' + \
            '?resource rdf:type dbo:Film . ' + \
            '?redirect dbo:wikiPageRedirects ?resource FILTER regex(?redirect, '  + '"' + str(film_year) + '") . } ' + \
        'UNION ' + \
            '{?resource rdfs:label '  + '"' + str(cell_content) + '" . ' + \
            '?resource rdf:type dbo:Film . ' + \
            '?resource rdfs:label ?name2 FILTER regex(?name2, '  + '"' + str(film_year) + '") . }' + \
        'UNION ' + \
            '{?resource rdfs:label ?name1 FILTER regex(?name1,'  + '"' + str(cell_content) + '", "i") . ' + \
            '?resource rdf:type dbo:Film . ' + \
            '?redirect dbo:wikiPageRedirects ?resource FILTER regex(?redirect, '  + '"' + str(film_year) + '") . } ' + \
        'UNION ' + \
            '{?resource rdfs:label ?name1 FILTER regex(?name1,'  + '"' + str(cell_content) + '", "i") . ' + \
            '?resource rdf:type dbo:Film . ' + \
            '?resource rdfs:label ?name2 FILTER regex(?name2, '  + '"' + str(film_year) + '") . }' + \
        'UNION ' + \
            '{?resource rdfs:label ?name1 FILTER regex(?name1,'  + '"' + str(cell_content) + '", "i") . ' + \
            '?resource rdf:type dbo:Film . }}'
results = send_query(query)
end = time.time()
print(end-start)
print(query)
# start = time.time()
# query1 = 'SELECT ?resource WHERE {' + \
#             '?resource rdfs:label ?name1 FILTER regex(?name1,'  + '"^' + str(cell_content) + '$", "i") . ' + \
#             '?resource rdf:type dbo:Film . }'

# query2 = 'SELECT ?resource WHERE {' + \
#             '?resource rdfs:label ?name1 FILTER regex(?name1,'  + '"^The ' + str(cell_content) + '$", "i") . ' + \
#             '?resource rdf:type dbo:Film . }'

# query3 = 'SELECT ?resource WHERE {' + \
#             '?resource rdfs:label '  + '"' + str(cell_content) + '" . ' + \
#             '?resource rdf:type dbo:Film . ' + \
#             '?redirect dbo:wikiPageRedirects ?resource FILTER regex(?redirect, '  + '"' + str(film_year) + '") . } '

# query4 = 'SELECT ?resource WHERE {' + \
#             '?resource rdfs:label '  + '"' + str(cell_content) + '" . ' + \
#             '?resource rdf:type dbo:Film . ' + \
#             '?resource rdfs:label ?name2 FILTER regex(?name2, '  + '"' + str(film_year) + '") . }' 

# query5 = 'SELECT ?resource WHERE {' + \
#             '?resource rdfs:label ?name1 FILTER regex(?name1,'  + '"' + str(cell_content) + '", "i") . ' + \
#             '?resource rdf:type dbo:Film . ' + \
#             '?redirect dbo:wikiPageRedirects ?resource FILTER regex(?redirect, '  + '"' + str(film_year) + '") . } '

# query6 = 'SELECT ?resource WHERE {' + \
#             '?resource rdfs:label ?name1 FILTER regex(?name1,'  + '"' + str(cell_content) + '", "i") . ' + \
#             '?resource rdf:type dbo:Film . ' + \
#             '?resource rdfs:label ?name2 FILTER regex(?name2, '  + '"' + str(film_year) + '") . }'

# query7 = 'SELECT ?resource WHERE {' + \
#             '{?resource rdfs:label ?name1 FILTER regex(?name1,'  + '"' + str(cell_content) + '", "i") . ' + \
#             '?resource rdf:type dbo:Film . }}'

# list_of_queries = [query1, query2, query3, query4, query5, query6, query7]
# for query in list_of_queries:
#     results = send_query(query)
#     uri = get_URI_from_results(results, query, verbose=0)
#     if uri != "NOT FOUND URI":
#         break


end = time.time()
print(end-start)

uri = get_URI_from_results(results, query, verbose=0)
print(uri)
ll = []
for i in range(len(results['results']['bindings'])):
    ll.append(results['results']['bindings'][i]['resource']['value'])
ll


In [2]:
ll = [1,55,22,66]
for i, x in enumerate(ll):
    print(i, x)

0 1
1 55
2 22
3 66


In [9]:
target = pd.read_csv("D:\Dokumenty\Zastosowanie Technologii Informatycznych\data\CEA_Round1_Targets.csv",
                    sep = ',',
                     skiprows=range(112,113), # 112 wadliwy wiersz
                    names=['TableID', 'ColumnID', 'RowID'])
target[:3]

,TableID,ColumnID,RowID
0,50245608_0_871275842592178099,0,154
1,39107734_2_2329160387535788734,1,32
2,22864497_0_8632623712684511496,0,227


In [ ]:
# jeżeli inna komówka jest filmem

# select ?r where {
# ?r rdfs:label ?name FILTER regex(?name, "Pearl Harbor", "i") .
# ?r rdf:type dbo:Film .
# }

# to

In [ ]:
table_id = '35188621_0_6058553107571275232'
df = pd.read_csv(f"D:\Dokumenty\Zastosowanie Technologii Informatycznych\data\CEA_Round1\{table_id}.csv",
                    sep=',')

In [8]:
def replace_characters(chars_to_replace_list_of_dicts, cell_contents_list, cell_content):
    # zamienia wszystkie znaki odpowiadające kluczowi np. '[?]' na to co w wartości ''
    # każdy znak zamienia osobno. Nie wyszukuje stringa '[?]', tylko każdy znak po kolei szukam i zamieniam
    for dicts in chars_to_replace_list_of_dicts:
        for old_chars, new_char in dicts.items():
            for old_char in old_chars:
                cell_content = cell_content.replace(old_char, new_char)
            if cell_content not in cell_contents_list:
                cell_contents_list.append(cell_content)
    return cell_contents_list

def create_query(cell_contents_list, cell_content):
    # pierwsze zapytanie - bierze zawartość sformatowanej komórki i wyszukuje linku,
    query = 'SELECT ?resource WHERE { { ?resource rdfs:label ' + '"' + str(cell_content) + '"' + "@en" + '}'

    for cell_content in cell_contents_list:
        query = query + ' UNION { ?resource rdfs:label ' + '"' + str(cell_content) + '"' + "@en" + '} '

    lang = 'en'
    try:
        lang = detect(cell_content)
    except:
        pass

    if lang != 'en':
        for cell_content  in cell_contents_list:
            query = query + ' UNION { ?resource rdfs:label ' + '"' + str(cell_content) + '"' + '@' + str(lang) + '} '
    query = query + '}'
    
    return query

def send_query(query):
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    # jeśli wystąpi błąd z połączeniem się z serwerem to próbujemy aż do skutku
    results = None
    while results is None:
        try:
            results = sparql.query().convert()
        except Exception as e:
#                 print("EXCEPTION:" + str(e))
#                 print("TAKI: " + str(cell_content))
            continue
    return results

def get_URI_from_results(results, query, verbose=0):
    # jeśli nic nie znaleziono no to trudno, nie mamy linku, wstawiamy NOT FOUND URI
    if str(results['results']['bindings']).find("http://dbpedia.org/resource/") == -1:
        # printy przydatne do sprawdzania jakich zawartości nam nie wyszukał,
        # trzeba analizowac i dodawać kolejne warunki, zmieniać cell_content na różne sposoby
        if verbose:
            print(table_id, column_id, row_id)
            print(cell_content_copy)
            print(query)    
        return 'NOT FOUND URI'


    # jeśli jednak mamy jakies linki to wyciągamy go z jsona,
    # linków może być więcej niż 1, aktualnie bierzemy pierwszy lepszy
    results_list = results['results']['bindings']
    for result_dict in results_list:
        uri = result_dict['resource']['value']
        match = re.match("http://dbpedia.org/resource/*", uri)

        # czasami pojawiają dziwne linki np. http://dbpedia.org/resource/Category:Max_Weber
        # my takich nie chcemy i musimy ich unikać
        # interesują nas tylko np.  http://dbpedia.org/resource/Max_Weber
        # więc to jest warunek pozbywajacy się wszystkich linków z Category:
        if match and uri[5:].find("Category:") == -1:
#                 answers_dict['URI'].append(str(uri)
#                 formatted_uri = 'http://dbpedia.org/resource/' + str(urllib.parse.quote(uri[28:]))
#                 formatted_uri = formatted_uri.replace('-','%E2%80%93')
            return uri
    
    return 'NOT FOUND URI'

In [21]:
start = time.time()

# słownik, z którego będzie tworzony wynikowy df
answers_dict = {"TableID":[],
               "ColumnID":[],
               "RowID":[],
               "URI":[]}

# zmienna, aby ograniczyć liczbę przetwarzanych wierszy, niżej masz 3 linijki skomentowane,
# odkomentowujesz je i ustawiasz sobię liczbę wierszy w tym warunku, aktualnie 1000
counter = 0
# zlicza ile linków nie zostało znalezionych w ogóle,
# tzn. po wysłaniu zapytania nie dostaliśmy żądnej odpwiedzi od sparql (wiersze oznaczone przez NOT FOUND URI)
not_found_uris = 0

for index, row in target.iterrows():
    try:
        counter = counter + 1
#         if counter == 100:
#             break


        table_id = row['TableID']
        column_id = row['ColumnID']
        row_id = row['RowID']
        
        # wczytanie odpowiedniej tabeli
        df = pd.read_csv(f"D:\Dokumenty\Zastosowanie Technologii Informatycznych\data\CEA_Round1\{table_id}.csv",
                    sep=',')
        

            
        # wyciągniecie wartości komórki, która nas interesuje
        cell_content = df.iloc[row_id-1:row_id, column_id:(column_id+1)].values[0][0]
        cell_content_copy = cell_content
        print(str(counter) + ' ' + cell_content)
        # usunięcie białych znaków z początku i końca stringa
        cell_content = cell_content.strip()
        
        # sprawdzenie czy zapytanie prawdopodobnie dotyczy filmu
        if len({'Title', 'Year'}.intersection(set(df.columns))) == 2:
            row = df[row_id-1:row_id]
            film_year = row['Year'].values[0]
                     
            
            cell_content_film = cell_content.replace(',','')
            cell_content_film = cell_content_film.replace('The','')
            cell_content_film = cell_content_film.replace(',','')
            cell_content_film = cell_content_film.strip()
            

            query1 = 'SELECT ?resource WHERE {' + \
                        '?resource rdfs:label ?name1 FILTER regex(?name1,'  + '"^' + str(cell_content) + '$", "i") . ' + \
                        '?resource rdf:type dbo:Film . }'

            query2 = 'SELECT ?resource WHERE {' + \
                        '?resource rdfs:label ?name1 FILTER regex(?name1,'  + '"^The ' + str(cell_content) + '$", "i") . ' + \
                        '?resource rdf:type dbo:Film . }'

            query3 = 'SELECT ?resource WHERE {' + \
                        '?resource rdfs:label '  + '"' + str(cell_content) + '" . ' + \
                        '?resource rdf:type dbo:Film . ' + \
                        '?redirect dbo:wikiPageRedirects ?resource FILTER regex(?redirect, '  + '"' + str(film_year) + '") . } '

            query4 = 'SELECT ?resource WHERE {' + \
                        '?resource rdfs:label '  + '"' + str(cell_content) + '" . ' + \
                        '?resource rdf:type dbo:Film . ' + \
                        '?resource rdfs:label ?name2 FILTER regex(?name2, '  + '"' + str(film_year) + '") . }' 

            query5 = 'SELECT ?resource WHERE {' + \
                        '?resource rdfs:label ?name1 FILTER regex(?name1,'  + '"' + str(cell_content) + '", "i") . ' + \
                        '?resource rdf:type dbo:Film . ' + \
                        '?redirect dbo:wikiPageRedirects ?resource FILTER regex(?redirect, '  + '"' + str(film_year) + '") . } '

            query6 = 'SELECT ?resource WHERE {' + \
                        '?resource rdfs:label ?name1 FILTER regex(?name1,'  + '"' + str(cell_content) + '", "i") . ' + \
                        '?resource rdf:type dbo:Film . ' + \
                        '?resource rdfs:label ?name2 FILTER regex(?name2, '  + '"' + str(film_year) + '") . }'

            query7 = 'SELECT ?resource WHERE {' + \
                        '{?resource rdfs:label ?name1 FILTER regex(?name1,'  + '"' + str(cell_content) + '", "i") . ' + \
                        '?resource rdf:type dbo:Film . }}'

            list_of_queries = [query1, query2, query3, query4, query5, query6, query7]
            found = False
            for id, query in enumerate(list_of_queries):
                results = send_query(query)
                uri = get_URI_from_results(results, query, verbose=0)
                if uri != "NOT FOUND URI":
                    answers_dict['TableID'].append(table_id)
                    answers_dict['ColumnID'].append(column_id)
                    answers_dict['RowID'].append(row_id)
                    answers_dict['URI'].append(uri)
                    found = True
                    break
            if found:
                continue

#                 if id+1 == len(list_of_queries):
                    

#             if uri != 'NOT FOUND URI':
#                 answers_dict['TableID'].append(table_id)
#                 answers_dict['ColumnID'].append(column_id)
#                 answers_dict['RowID'].append(row_id)
#                 answers_dict['URI'].append(uri)
#                 continue
                    

        # aktualnie nie używane, ale pewnie będzie
        # pos_tag() - kategoryzuje słowa, np. znajduje czasowniki, rzeczowniki, nazwy własne (jak imiona i nazwiska)
        # chodzi o to, że czasami odpowiedzi są w angielskiej dbpedii,
        # np. Alex Rodriguez wykrywa jako język ca - pewnie kataloński, 
        # ale nie ma jego odpowiednika w katalońskiej dbpedii, a w angielskiej jest
        # dlatego chcę wykrywać nazwy wkłasne jak imiona i nazwiska i wtedy zawsze szukać w angielksiej dbpedii,
        # ale z tym też był problem czasami i na razie nie myślałem nad tym dalej
        # NNP - to kategoria nazwy własnej, np. imię
#         tags = pos_tag(cell_content.split())
#         is_one_NNP = False
#         for tag in tags:
#             if tag[1] == "NNP":
#                 is_one_NNP = True
#                 break
#         try:
#             if is_one_NNP:
#                 lang = '@en'
#             else:
#                 lang = str('@') + str(detect(cell_content))
#         except Exception:
#             lang = ''


        
        cell_contents_list = []
        cell_contents_list.append(cell_content) # oryginalne
        cell_contents_list.append(str('The ' + cell_content)) # opcja z The na początku
        cell_contents_list.append(cell_content.capitalize())
        cell_contents_list.append(cell_content.title())

        # pojawiały się wartości typy "Guardian The", a powinno być "The Gurdian" to samo z A i An
        if re.search(" The$", cell_content):
            index = cell_content.find(" The")
            cell_contents_list.append("The " + cell_content[:index].replace(',','').strip())
            cell_contents_list.append(cell_content[:index].strip())
        elif re.search(" A$", cell_content):
            index = cell_content.find(" A")
            cell_contents_list.append("A " + cell_content[:index].replace(',','').strip())
            cell_contents_list.append(cell_content[:index].strip())
        elif re.search(" An$", cell_content):
            index = cell_content.find(" An")
            cell_contents_list.append("An " + cell_content[:index].replace(',','').strip())
            cell_contents_list.append(cell_content[:index].strip())
            
        # usunięcię The z początku
        if re.search("^The ", cell_content):
            cell_contents_list.append(re.sub('^The ', '', cell_content))
       
    
        strings_to_replace_dict = [{'[?]':''}, {',':''}, {'-':' '}, {'?':''}] # dict = {chars_to_replace:new_char}
        cell_contents_list = replace_characters(strings_to_replace_dict, cell_contents_list, cell_content)

        
        query = create_query(cell_contents_list, cell_content)
        results = send_query(query)                              
        uri = get_URI_from_results(results, query, verbose=1) 
                                  
        answers_dict['TableID'].append(table_id)
        answers_dict['ColumnID'].append(column_id)
        answers_dict['RowID'].append(row_id)
        answers_dict['URI'].append(uri)
        if uri == 'NOT FOUND URI':
            not_found_uris += 1
            continue

    except Exception as e:
        # na początku były problemy z danymi, naprawiłem
        # potem zaczęły pojawiać sę problemy, żę serwer czasami nie odpowiadał,
        # więc jak jest error to olewamy i przechodzimy do następnego wiersza
        # będzie trzeba zrobić, że jak serwer nie odpowie z jakiegoś powodu to trzeba ponowić to samo zapytanie
        # zamiast pomijać, bo tracimy wiersze
        exc_type, exc_obj, exc_tb = sys.exc_info()
        print(table_id, column_id, row_id) # jaki plik, kolumna i wiersz
        print(cell_content_copy) # zawartość komórki
        print(e) # jaki błąd, teraz chyba tylko z brakiem połączenia z serwerem
        print(exc_tb.tb_lineno) # w jakiej liniii błąd
        continue



# print(answers_dict)
result_df = pd.DataFrame(answers_dict)
end = time.time()
print(end-start)
print(not_found_uris)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 96)

In [13]:
result_df_copy = result_df.copy()

In [18]:
result_df_copy.shape

(8415, 4)

In [17]:
result_df_copy = result_df_copy.drop_duplicates(subset=['TableID','ColumnID','RowID'], keep="first")
result_df_copy[:20]

,TableID,ColumnID,RowID,URI
0,50245608_0_871275842592178099,0,154,http://dbpedia.org/resource/Pearl_Harbor_(film)
2,39107734_2_2329160387535788734,1,32,http://dbpedia.org/resource/Minneapolis-Saint_...
3,22864497_0_8632623712684511496,0,227,http://dbpedia.org/resource/Need_for_Speed:_Ca...
4,66009064_0_9148652238372261251,0,15,http://dbpedia.org/resource/Alex_Rodriguez
5,21362676_0_6854186738074119688,1,75,http://dbpedia.org/resource/King_Kong_(1933_film)
7,50245608_0_871275842592178099,0,186,http://dbpedia.org/resource/The_School_of_Rock
8,40534006_0_4617468856744635526,1,1,http://dbpedia.org/resource/Lex_Luger
9,36102169_0_7739454799295072814,2,286,http://dbpedia.org/resource/Crash_Bash
10,53822652_0_5767892317858575530,1,141,http://dbpedia.org/resource/The_Hurt_Locker
11,60319454_0_3938426910282115527,0,29,http://dbpedia.org/resource/Lion


In [11]:
print(result_df.shape)
print(target.shape)
result_df[:20]
# result_df[10:13]

(9990, 4)
(8417, 3)


,TableID,ColumnID,RowID,URI
0,50245608_0_871275842592178099,0,154,http://dbpedia.org/resource/Pearl_Harbor_(film)
1,50245608_0_871275842592178099,0,154,http://dbpedia.org/resource/Pearl_Harbor
2,39107734_2_2329160387535788734,1,32,http://dbpedia.org/resource/Minneapolis-Saint_...
3,22864497_0_8632623712684511496,0,227,http://dbpedia.org/resource/Need_for_Speed:_Ca...
4,66009064_0_9148652238372261251,0,15,http://dbpedia.org/resource/Alex_Rodriguez
5,21362676_0_6854186738074119688,1,75,http://dbpedia.org/resource/King_Kong_(1933_film)
6,21362676_0_6854186738074119688,1,75,http://dbpedia.org/resource/King_Kong
7,50245608_0_871275842592178099,0,186,http://dbpedia.org/resource/The_School_of_Rock
8,40534006_0_4617468856744635526,1,1,http://dbpedia.org/resource/Lex_Luger
9,36102169_0_7739454799295072814,2,286,http://dbpedia.org/resource/Crash_Bash


In [19]:
# zapisanie submission
# przez evaluator codes można sprawdzić wyniki
result_df.to_csv("D:\Dokumenty\Zastosowanie Technologii Informatycznych\evaluator_codes\evaluator_codes\data\my_submission_films2.csv",
                 index=False,
                 sep=',',
                header=False)

### NIEISTOTNE, TESTY ITP.

In [ ]:
df = pd.read_csv("D:\\Dokumenty\\Zastosowanie Technologii Informatycznych\\data\\CEA_Round1\\13719111_1_5719401842463579519.csv",
                sep=',')
df[:3]

In [ ]:
detect(df.iloc[0:1, 1:2].values[0][0])

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?label
    WHERE { <http://dbpedia.org/resource/Norway> rdfs:label ?label }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print(results)
for result in results["results"]["bindings"]:
    print(result["label"]["value"])

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX ont: <http://dbpedia.org/ontology/>
    SELECT ?a, ?c
    WHERE { 
            ?a ont:location <http://dbpedia.org/resource/Delft> . 
            FILTER regex(?a, "Ikea", "i")
          }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print(results)
# print(result['a'])
# print(result['b'])
# print(result['c'])

# for result in results["results"]["bindings"]:
#     print(type(result))

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sss = "dbr:Game_of_Thrones"
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery('''
ASK {e}
    VALUES (?r) {e} ({source}) {w}
        {e} ?r ?p ?o {w}
        UNION
        {e} ?s ?r ?o {w}
        UNION
        {e} ?s ?p ?r {w}
    {w} 
'''.format(source=sss, e="{", w="}"))
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print(results)

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sss = "dbr:Game_of_Thrones"
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery('''
ASK {
    VALUES (?r) { (dbr:Gra_o_tron) }
        { ?r ?p ?o }
        UNION
        { ?s ?r ?o }
        UNION
        { ?s ?p ?r }
    } 
''')
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print(results)



In [ ]:
SELECT ?r ?p ?o
WHERE {
 VALUES (?r) { (dbr:Cat) }
        { ?r ?p ?o }
        UNION
        { ?s ?r ?o }
        UNION
        { ?s ?p ?r }
}

select ?s
where {
        { ?s rdfs:label "Kot domowy"@pl }
        UNION
        { "Kot domowy"@pl rdfs:label ?s }
} 

SELECT ?r
WHERE {
        { ?r rdfs:label "Kaspisches Meer"@de }

}

In [ ]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    SELECT ?resource
    WHERE { ?resource rdfs:label "Kaspisches Meer"@de }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
# print(results['results']['bindings'][1]['r']['value'])
# results = str(results['results']['bindings'])
print(str(results['results']['bindings']))
if str(results['results']['bindings']).find("http://dbpedia.org/resource/") == -1:
    print("nie ma")
else:
    print('jest')
# for result_dict in results_list:
#     uri = result_dict['resource']['value']
#     match = re.match("http://dbpedia.org/resource/*", uri)
#     if match:
#         print(uri)
#     else:
#         print("NIE")

# print(results['results']['bindings'])

In [ ]:
import re
match = re.match("http://dbpedia.org/resource/*",'http://dbpedia.org/resource/Caspian_Sea', re.M|re.I)
if match:
    print(match.group())